##### Copyright 2018 The TensorFlow Authors.


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/text"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/text.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates two ways to load and preprocess text.

- First, you will use Keras utilities and preprocessing layers. These include `tf.keras.utils.text_dataset_from_directory` to turn data into a `tf.data.Dataset` and `tf.keras.layers.TextVectorization` for data standardization, tokenization, and vectorization. If you are new to TensorFlow, you should start with these.
- Then, you will use lower-level utilities like `tf.data.TextLineDataset` to load text files, and [TensorFlow Text](https://www.tensorflow.org/text) APIs, such as `text.UnicodeScriptTokenizer` and `text.case_fold_utf8`, to preprocess the data for finer-grain control.

In [1]:
# !pip install "tensorflow-text==2.8.*"

^C


In [2]:
import collections
import pathlib
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
# import tensorflow_text as tf_text

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.10.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


## Example 1: Predict IMDB Reviews

### Download and explore the dataset

In [3]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
raw_train_ds, raw_val_ds, raw_test_ds = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    shuffle_files=True,
    as_supervised=True)

In [4]:
raw_train_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [5]:
text_batch, label_batch = next(iter(raw_train_ds.batch(3)))
for i in range(3):
    print (text_batch.numpy()[i])
    print (label_batch.numpy()[i])

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
0
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development w

### Prepare the dataset for training

Next, you will standardize, tokenize, and vectorize the data using the `tf.keras.layers.TextVectorization` layer.

- _Standardization_ refers to preprocessing the text, typically to remove punctuation or HTML elements to simplify the dataset.
- _Tokenization_ refers to splitting strings into tokens (for example, splitting a sentence into individual words by splitting on whitespace).
- _Vectorization_ refers to converting tokens into numbers so they can be fed into a neural network.

All of these tasks can be accomplished with this layer. (You can learn more about each of these in the `tf.keras.layers.TextVectorization` API docs.)

Note that:

- The default standardization converts text to lowercase and removes punctuation (`standardize='lower_and_strip_punctuation'`).
- The default tokenizer splits on whitespace (`split='whitespace'`).
- The default vectorization mode is `'int'` (`output_mode='int'`). This outputs integer indices (one per token). This mode can be used to build models that take word order into account. You can also use other modes—like `'binary'`—to build [bag-of-words](https://developers.google.com/machine-learning/glossary#bag-of-words) models.

You will build two models to learn more about standardization, tokenization, and vectorization with `TextVectorization`:

- Then, you will use the `'int'` mode with a 1D ConvNet.

For the `'int'` mode, in addition to maximum vocabulary size, you need to set an explicit maximum sequence length (`MAX_SEQUENCE_LENGTH`), which will cause the layer to pad or truncate sequences to exactly `output_sequence_length` values:

In [6]:
MAX_SEQUENCE_LENGTH = 250
VOCAB_SIZE = 10000

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

Next, call `TextVectorization.adapt` to fit the state of the preprocessing layer to the dataset. This will cause the model to build an index of strings to integers.

Note: It's important to only use your training data when calling `TextVectorization.adapt`, as using the test set would leak information.

In [7]:
# Make a text-only dataset (without labels), then call `TextVectorization.adapt`.
train_text = raw_train_ds.map(lambda text, labels: text)
int_vectorize_layer.adapt(train_text)

In [20]:
train_text

<MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

Print the result of using these layers to preprocess data:

In [9]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [10]:
# Retrieve a batch (of 32 reviews and labels) from the dataset.
text_batch, label_batch = next(iter(raw_train_ds.batch(2)))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

Question tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
Label tf.Tensor(0, shape=(), dtype=int64)


In [11]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])

'int' vectorized question: tf.Tensor(
[[  11   14   34  423  375   18   89   27 9105    8   33 1273 3859   41
   509    1  194   25   85  153   19   11  213  316   27   66  240  218
     8  485   55   66   85  113   95   22 6196   11   92  674  747   11
    18    7   34  397 9079  170 2409  407    2   88 1174  135   67  144
    52    2    1 7610   67  247   66 2925   16    1 2620    1    1 1426
  4609    3   40    1 1668   17 3859   14  159   19    4 1174  866 7620
     8    4   18   12   14 3836    5   98  146 1175   10  228  694   12
    48   25   92   39   11 7811  153   39 1273    1   50  406   10   95
  1162  825  140    9    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0

As shown above, `TextVectorization`'s `'binary'` mode returns an array denoting which tokens exist at least once in the input, while the `'int'` mode replaces each token by an integer, thus preserving their order.

You can lookup the token (string) that each integer corresponds to by calling `TextVectorization.get_vocabulary` on the layer:

In [12]:
print("1289 ---> ", int_vectorize_layer.get_vocabulary()[1289])
print("313 ---> ", int_vectorize_layer.get_vocabulary()[313])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

1289 --->  critics
313 --->  special
Vocabulary size: 10000


You are nearly ready to train your model.

As a final preprocessing step, you will apply the `TextVectorization` layers you created earlier to the training, validation, and test sets:

In [13]:
int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

### Configure the dataset for performance

These are two important methods you should use when loading data to make sure that I/O does not become blocking.

- `Dataset.cache` keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache, which is more efficient to read than many small files.
- `Dataset.prefetch` overlaps data preprocessing and model execution while training.

You can learn more about both methods, as well as how to cache data to disk in the *Prefetching* section of the [Better performance with the tf.data API](../../guide/data_performance.ipynb) guide.

In [14]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [15]:
int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

### Train the model

It's time to create your neural network.


Next, you will use the `'int'` vectorized layer to build a 1D ConvNet:

In [217]:
embed_size = 250

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(VOCAB_SIZE, embed_size))
model.add(tf.keras.layers.LSTM(units=250, activation='tanh'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
model.add(tf.keras.layers.Flatten())

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [218]:
model.summary()

Model: "sequential_73"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_51 (Embedding)    (None, None, 250)         2500000   
                                                                 
 lstm_50 (LSTM)              (None, 250)               501000    
                                                                 
 dense_70 (Dense)            (None, 1)                 251       
                                                                 
 flatten_16 (Flatten)        (None, 1)                 0         
                                                                 
Total params: 3,001,251
Trainable params: 3,001,251
Non-trainable params: 0
_________________________________________________________________


In [219]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=3)

Epoch 1/3


ValueError: in user code:

    File "c:\Users\DPERALES\Anaconda3\envs\testwebapps\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\DPERALES\Anaconda3\envs\testwebapps\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\DPERALES\Anaconda3\envs\testwebapps\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\DPERALES\Anaconda3\envs\testwebapps\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\DPERALES\Anaconda3\envs\testwebapps\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\DPERALES\Anaconda3\envs\testwebapps\lib\site-packages\ker

In [ ]:
test_loss, test_acurracy = model.evaluate(X_test, y_test)

In [23]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      # layers.Embedding(VOCAB_SIZE, 64, mask_zero=True),
      # layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      # layers.GlobalMaxPooling1D(),
      # layers.Dense(num_labels)
      layers.GlobalAveragePooling1D(),
      layers.Dense(64, activation='relu'),
      layers.Dense(1, activation='sigmoid')
  ])
  return model

In [24]:
# `vocab_size` is `VOCAB_SIZE + 1` since `0` is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds.shuffle(10000).batch(256), validation_data=int_val_ds.batch(256), epochs=5)

Epoch 1/5
59/59 [==============================] - 1s 10ms/step - loss: 368.2610 - accuracy: 0.5060 - val_loss: 220.0347 - val_accuracy: 0.5003
Epoch 2/5
59/59 [==============================] - 0s 6ms/step - loss: 173.7911 - accuracy: 0.5207 - val_loss: 167.3782 - val_accuracy: 0.5037
Epoch 3/5
59/59 [==============================] - 0s 6ms/step - loss: 125.4946 - accuracy: 0.5415 - val_loss: 136.8411 - val_accuracy: 0.5047
Epoch 4/5
59/59 [==============================] - 0s 6ms/step - loss: 93.1627 - accuracy: 0.5545 - val_loss: 112.4402 - val_accuracy: 0.5007
Epoch 5/5
59/59 [==============================] - 0s 7ms/step - loss: 69.1017 - accuracy: 0.5733 - val_loss: 93.6766 - val_accuracy: 0.5043


Compare the two models:

In [19]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())

ConvNet model on int vectorized data:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 global_average_pooling1d (G  (None, 250)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 64)                16064     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 16,129
Trainable params: 16,129
Non-trainable params: 0
_________________________________________________________________
None


Evaluate both models on the test data:

In [20]:
int_loss, int_accuracy = int_model.evaluate(int_test_ds.shuffle(10000).batch(256))

print("Int model accuracy: {:2.2%}".format(int_accuracy))

98/98 [==============================] - 13s 77ms/step - loss: 87.4833 - accuracy: 0.5034
Int model accuracy: 50.34%


Note: This example dataset represents a rather simple classification problem. More complex datasets and problems bring out subtle but significant differences in preprocessing strategies and model architectures. Be sure to try out different hyperparameters and epochs to compare various approaches.